In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using GLMakie
using LinearAlgebra, LinearSolve,Statistics

cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [2]:
function condition(u, t, integrator)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf) - ϵ_box
end
affect!(integrator) = terminate!(integrator)
function condition(u)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf), linsolve
end
function color_alpha(alpha)
    if alpha > 0.0
        return "red"
    else
        return "blue"
    end
end;

In [3]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305

t = 500.0; tspan = [0.0, t];

In [22]:
# index point from homoclinic curve
index_hom = 1;

I0 = I0_hom[index_hom]; U0 = u0_hom[index_hom]

p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [11.325905642223786, 0.6594706953104683, 0.4863175026548461]

ds = CoupledODEs(TM, u0, p)
fp, _, _ = fixedpoints(ds, box, jacob_TM_)

ϵ_box = 1e-4
Ju0 = jacob_TM_(fp[1], p, 0);
fp_ = fp[1]

eigen_val_vec = eigen(Ju0)
eigen_vectors = eigen_val_vec.vectors

v1 = real(eigen_vectors[:, 1])
v2 = real(eigen_vectors[:, 2])
v3 = imag(eigen_vectors[:, 3])

v1 = normalize(v1, 2)
v2 = normalize(v2, 2)
v3 = normalize(v3, 2)

v1t = transpose(v1)
v2t = transpose(v2)
v3t = transpose(v3)

A = transpose([v1t; v2t; v3t])
A = Matrix(A)

N = 101
nrange = range(1, N - 1, step = 1)

dots = zeros(length(nrange), 3)
Es = Float64[]; xs = Float64[]; ys = Float64[]
α1s =  Float64[]; α2s =  Float64[]; α3s =  Float64[]

colors = []
check_events = []
time_events = []
"""for (i, n) in enumerate(nrange)
    α1 = 0
    α2 = ϵ_box
    α3 = -ϵ_box + 2 * ϵ_box * n / N
    αs = [α1, α2, α3]
    dot = fp_ + A * αs
    dots[i, :] = dot
end""";

In [23]:
N = 101 
n_total = N - 1
n_line = Int( n_total/ 4 ) # точек на одной прямой


Float64[]

In [26]:
index = 1
while index != n_line
    α1 = 0
    α2 = ϵ_box
    α3 = -ϵ_box + 2 * ϵ_box * index / N
    αs = [α1, α2, α3]
    dot = fp_ + A * αs
    push!(Es, dot[1])
    push!(xs, dot[2])
    push!(ys, dot[3])
    index += 1
end
index = index

25

In [28]:
for i in range(1, n_line, step = 1)
    _, αs = condition([ Es[i], xs[i], ys[i] ])
    push!(α1s, αs[1])
    push!(α2s, αs[2])
    push!(α3s, αs[3])
end

BoundsError: BoundsError: attempt to access 24-element Vector{Float64} at index [25]

In [21]:
# last points in phase space

f = Figure(resolution = (700, 700))

ax = Axis(f[1, 1])

scatter!(ax, α2s, α3s, markersize=  5.0)

display(GLMakie.Screen(), f);